In [2]:
import time
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC 

### list of url of state transport that are needed to be scraped

In [30]:
url_lis=[['APSRTC','https://www.redbus.in/online-booking/apsrtc/?utm_source=rtchometile',6],
         ['TSRTC','https://www.redbus.in/online-booking/tsrtc/?utm_source=rtchometile',4],
         ['KSRTC','https://www.redbus.in/online-booking/ksrtc-kerala/?utm_source=rtchometile',3],
         ['SBSTC','https://www.redbus.in/online-booking/south-bengal-state-transport-corporation-sbstc/?utm_source=rtchometile',5],
         ['WBSTC','https://www.redbus.in/online-booking/west-bengal-transport-corporation?utm_source=rtchometile',2],
         ['BSRTC','https://www.redbus.in/online-booking/bihar-state-road-transport-corporation-bsrtc/?utm_source=rtchometile',5],
         ['HRTC','https://www.redbus.in/online-booking/hrtc/?utm_source=rtchometile',6],
         ['RSRTC','https://www.redbus.in/online-booking/rsrtc/?utm_source=rtchometile',4],
         ['PEPSU','https://www.redbus.in/online-booking/pepsu/?utm_source=rtchometile',4],
         ['ASTC','https://www.redbus.in/online-booking/astc/?utm_source=rtchometile',6],
         ['WBTC','https://www.redbus.in/online-booking/wbtc-ctc/?utm_source=rtchometile',6],
         ['KTCL','https://www.redbus.in/online-booking/ktcl/?utm_source=rtchometile',5]]

### working model for mutiple url to scrape routes of public transport

In [ ]:
# working model for mutiple url
# code for scarping main routes name of particular transport along with price and time
driver = webdriver.Chrome()

sta_trans,routes,route_link,bus_tim,first_bus,last_bus,st_pri=([] for f in range(7))

for h in range(len(url_lis)):
    
    driver.get(f'{url_lis[h][1]}')

    time.sleep(5)
    driver.maximize_window()
    driver.execute_script('window.scrollBy(0, 2000)')
    time.sleep(2)
# code for scraping first page

    ro_li=driver.find_elements(By.CLASS_NAME,'route')
    st=driver.find_elements(By.XPATH,"//span[@class='fare']")
    tim=driver.find_elements(By.CSS_SELECTOR,"span[class='totalRoutes']>strong")

    for j,k in zip(ro_li,st):
        routes.append(j.text)
        n=j.get_attribute('href')
        route_link.append(n)
        st_pri.append(k.text)
        sta_trans.append(url_lis[h][0])

    for l in range(len(tim)):
        if l==0 or l%2==0:
            g=tim[l].text
            first_bus.append(g)
        else:
            g=tim[l].text
            last_bus.append(g)  
         # using sep for loop since both times are getting scraped

# creating for loop to scrape other than first page
    
    for m in range(2,url_lis[h][2]):
        
        driver.find_element(By.XPATH,f"//div[contains(text(),'{m}')]").click()
        time.sleep(5)

        ro_li=driver.find_elements(By.CLASS_NAME,'route')
        st=driver.find_elements(By.XPATH,"//span[@class='fare']")
        tim=driver.find_elements(By.CSS_SELECTOR,"span[class='totalRoutes']>strong")
        
        for j,k in zip(ro_li,st):
            routes.append(j.text)
            n=j.get_attribute('href')
            route_link.append(n)
            st_pri.append(k.text)
            sta_trans.append(url_lis[h][0])
            
        for l in range(len(tim)):
            if l==0 or l%2==0:
                g=tim[l].text
                first_bus.append(g)
            else:
                g=tim[l].text
                last_bus.append(g)   # using sep for loop since the tim list lengths are twice than ro,st
    
ap={'State_Transport':sta_trans,'Routes_Name':routes,'Route_Link':route_link,'First_Bus':first_bus,'Last_Bus':last_bus,'Starting_Price':st_pri}

df=pd.DataFrame(ap)
print(df.to_markdown())

df.to_csv(r'C:\Users\Jayas\OneDrive\Desktop\redbus project\entire_transport_routes.csv',index=False)

### before using above links for data scraping check for duplicates use redbus_dataframe_correctiion.ipynb file for that

In [ ]:
entire_transport_routes=pd.read_csv('C:/Users/Jayas/OneDrive/Desktop/redbus project/entire_transport_routes.csv',index_col=[0])
entire_transport_routes

#### use it to read entire_transport_routes for correction

In [ ]:
b=entire_transport_routes[entire_transport_routes.duplicated(subset=["Route_Link"],keep=False)]
b.sort_values('Route_Link')
print(a.to_markdown())

a=entire_transport_routes[entire_transport_routes.duplicated(subset=["Route_Link"])]
a=a.sort_values('Route_Link')
print(a.to_markdown())

#### use if any duplicates found to remove and save it in csv

In [ ]:
entire_transport_routes=entire_transport_routes[~entire_transport_routes.duplicated(subset=["Route_Link"],keep='first')]
entire_transport_routes.to_csv(r'C:\Users\Jayas\OneDrive\Desktop\redbus project\entire_transport_routes.csv',index=[False])

### use to load urls for scraping bus details

In [ ]:
entire_transport_routes=pd.read_csv('C:/Users/Jayas/OneDrive/Desktop/redbus project/entire_transport_routes.csv',index_col=[0])
entire_transport_routes

In [ ]:
url_og=list(entire_transport_routes['Route_Link'])
print(url_og,len(url_og))

In [ ]:
url=url_og[310:] 
url

### working model for mutiple url to scrape routes of public transport

In [64]:
# working model for mutiple url

#creating list to store values collect from scraped selenium variable 
bus_count,starting_place,reaching_place,route_link,bus_name,bus_type,departing_time,duration,reaching_time,price,star_rating,no_reviews,seat_availability,vacant_type=([] for i in range(14))

#function for appending values by iterating selinium values to list with NAN value finder logic
def fun_append(sl,li):
    for i in sl:
        iText=i.text
        if not iText.strip(): li.append('NAN')
        else: li.append(i.text)

#function spliting and then appending values by iterating selinium values to list with NAN value finder logic
def fun_split_append(sl,li1,li2):
    for n in sl:
        nText=n.text
        if not nText.strip(): 
            li1.append('NAN')
            li2.append('NAN')
        else:
            n=str(n.text)
            if '\n' in n:
                n=n.split('\n')
                li1.append(n[0])
                li2.append(n[1])
            else:
                li1.append(n)
                li2.append('NAN')

driver=webdriver.Chrome()

#first for loop to get multiple urls
for q in range(len(url)):   
    
    driver.get(f'{url[q]}')
    time.sleep(5)
    driver.maximize_window()
    driver.execute_script('window.scrollBy(0, 200)')
    time.sleep(3)

    st_pla=driver.find_element(By.XPATH,("//*[@id='fixer']/div/div/div[1]/div/span[1]")) # data scraping
    rea_pla=driver.find_element(By.XPATH,("//*[@id='fixer']/div/div/div[1]/div/span[2]")) # data scraping

    # clicking nesscery buttons and scrolling down to load entire page data for lazyloading
    try:
        WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH,"(//*[@id='result-section']/div[2]/div/div[3]/div/div[4]/div[2])"))).click()
    except:
        pass
    try:
        WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH,"(//*[@id='result-section']/div[2]/div/div[2]/div/div[4]/div[2])"))).click()
    except:
        pass    
    try:
        WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH,"(//*[@id='result-section']/div[1]/div/div[2]/div/div[4]/div[2])"))).click()
    except:
        pass
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1)  # Allow time for more content to load
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    driver.execute_script('window.scrollBy(0, -9500)')
    time.sleep(5)
    #scrolling finishes here
    
    try:
        buses=driver.find_element(By.XPATH,("//*[@id='root']/div/div[2]/div/div[2]/div[1]/div[1]/span[1]/span"))
        bus_nam=driver.find_elements(By.XPATH,"(//div[@class='travels lh-24 f-bold d-color'])")
        bus_ty=driver.find_elements(By.XPATH,"(//div[@class='bus-type f-12 m-top-16 l-color evBus'])")
        de_ti=driver.find_elements(By.XPATH,"(//div[@class='dp-time f-19 d-color f-bold'])")
        durati=driver.find_elements(By.XPATH,"(//div[@class='dur l-color lh-24'])")
        re_ti=driver.find_elements(By.XPATH,"(//div[@class='bp-time f-19 d-color disp-Inline'])")
        pri=driver.find_elements(By.XPATH,"(//div[@class='fare d-block'])")

        st_rt=driver.find_elements(By.XPATH,"(//div[@class='column-six p-right-10 w-10 fl'])")
        se_av=driver.find_elements(By.XPATH,"(//div[@class='column-eight w-15 fl'])")
            
        bus_count.append(buses.text)# getting bus count per webpage to verify all bus details are scraped

        #1)getting nan value if text not present
        seli_list=[bus_nam,bus_ty,de_ti,durati,re_ti,pri]
        list_f_list=[bus_name,bus_type,departing_time,duration,reaching_time,price]
                
        for j in range(len(seli_list)):
            fun_append(seli_list[j],list_f_list[j])

        #1)getting nan value if text not found 2) seprating rating and number of reviewed persons and review and appending only rating
        seli=[st_rt,se_av]
        lis1=[star_rating,seat_availability]
        lis2=[no_reviews,vacant_type]

        for k in range(len(seli)):
            fun_split_append(seli[k],lis1[k],lis2[k])

        # removing bus in route name and making list of route name 
                
        for l in range(len(bus_nam)):
            l=url[q]
            route_link.append(l)
        for m in range(len(bus_nam)):
            starting_place.append(st_pla.text)
        for n in range(len(bus_nam)):
            reaching_place.append(rea_pla.text)
        
    except:
        pass

di={'Starting_Place':starting_place,'Reaching_Place':reaching_place,'Route_Link':route_link,'Bus_Name':bus_name,'Bus_Type':bus_type,
    'Departing_Time':departing_time,'Duration':duration,'Reaching_Time':reaching_time,
    'Star_Rating':star_rating,'Number_of_Reviews':no_reviews,'Price':price,'Seat_Availability':seat_availability,'Vacant_Type':vacant_type}

gp2=pd.DataFrame(di)

print(gp2.to_markdown()) # to view entire dataframe



|     | Starting_Place      | Reaching_Place      | Route_Link                                                        | Bus_Name                                                                          | Bus_Type                                         | Departing_Time   | Duration   | Reaching_Time   | Star_Rating   |   Number_of_Reviews | Price    | Seat_Availability   | Vacant_Type   |
|----:|:--------------------|:--------------------|:------------------------------------------------------------------|:----------------------------------------------------------------------------------|:-------------------------------------------------|:-----------------|:-----------|:----------------|:--------------|--------------------:|:---------|:--------------------|:--------------|
|   0 | Durgapur (West B... | Purulia             | https://www.redbus.in/bus-tickets/durgapur-to-purulia             | SBSTC-KOLKATA - BAGMUNDI - 08:30 (BEL DEPOT) - 480                                | Non AC Seate

In [65]:
# use to make sure all bus details are scrapped or not
b=[]
for i in bus_count:
    i=i.strip('Buses')
    i=int(i)
    b.append(i)
sum(b)    

439

In [ ]:
                         # use it to verify the list length incase of error
         #    ERROR: ALL ARRAY SHOULD BE OF SAME LENGTH

#len(starting_place),len(reaching_place),len(route_link),len(bus_name),len(bus_type),len(departing_time),len(duration),len(reaching_time),len(star_rating),len(no_reviews),len(price),len(seat_availability)

### process below is part by part storing data to entire_bus_details 

In [22]:
# USE IT TO CREATE EMPTY DATAFRAME TO USE IN CONCATENATION 
#entire_bus_details=pd.DataFrame(columns=['Starting_Place','Reaching_Place','Route_Link','Bus_Name','Bus_Type','Departing_Time','Duration','Reaching_Time',
#                                 'Star_Rating','Number_of_Reviews','Price','Seat_Availability','Vacant_Type'])

In [66]:
scraped = gp2 # getting gp2 dataframe which is created in scraping process into scraped dataframe

In [3]:
# reading main csv file incase of new session

#entire_bus_details=pd.read_csv('C:/Users/Jayas/OneDrive/Desktop/redbus project/entire_bus_details.csv',index_col=[0])

In [67]:
entire_bus_details.shape # use to check row size befoe concat

(13518, 13)

In [68]:

entire_bus_details=pd.concat([entire_bus_details,scraped],ignore_index=True)

entire_bus_details.to_csv(r'C:\Users\Jayas\OneDrive\Desktop\redbus project\entire_bus_details.csv',index=[False])


In [69]:
entire_bus_details.shape # use to check row size before concat

(13957, 13)

In [ ]:
entire_bus_details.isna().sum()

In [ ]:
print(entire_bus_details.to_markdown())

### use for data correction of entire_bus_details csv file

In [ ]:
entire_bus_details=pd.read_csv('C:/Users/Jayas/OneDrive/Desktop/redbus project/entire_bus_details.csv',index_col=[0])
entire_bus_details.info()

In [ ]:
entire_bus_details['Price'] = entire_bus_details['Price'].str.replace("INR","")
entire_bus_details['Seat_Availability'] = entire_bus_details['Seat_Availability'].str.strip(" Seat available")

In [ ]:
entire_bus_details.to_csv(r'C:\Users\Jayas\OneDrive\Desktop\redbus project\entire_bus_details.csv',index=[False])